# Backend: Germany Power Generation – Clean, Step-by-Step Solution

**Scope:** Hourly electricity generation by fuel for Germany (Oct–Dec 2022), cleaned & aggregated.  
**Deliverables:** Cleaned hourly table (wide), daily & monthly aggregates, renewable vs non-renewable tables, and figures.  
**Run order:** Execute cells from top to bottom. If the API is unavailable, the notebook will look for a local file in `./data/`.

> **Rename this notebook to** `backend_<first_name>_<last_name>.ipynb` before submission.
THIS WAS A GUIDE B CHAT GPT PLEASE dont USE THIS

In [ ]:
# --- 0. Setup & Imports ---
# If you don't have some packages, install them with:
# %pip install pandas numpy matplotlib requests python-dotenv pyarrow

import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path

pd.set_option("display.max_columns", None)
plt.rcParams["figure.figsize"] = (12, 6)

DATA_DIR = Path("./data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# Optional: load API credentials from a .env file placed next to this notebook
# .env example:
#   BASE_URL=https://<provided-api-endpoint>/query
#   API_KEY=xxxxxxxxxxxxxxxxxxxxxxxx
try:
    from dotenv import load_dotenv  # type: ignore
    load_dotenv()
except Exception:
    pass

BASE_URL = os.getenv("BASE_URL", "").strip()  # provided by organizer
API_KEY  = os.getenv("API_KEY", "").strip()

DATASET  = "task_generation_h"
DATE_FROM = "2022-10-01"
DATE_TO   = "2022-12-31"

# Local cache filenames (used when API is down or to avoid re-downloading)
RAW_JSON_PATH = DATA_DIR / "raw_generation.json"
CLEAN_HOURLY_WIDE_CSV = DATA_DIR / "germany_hourly_by_fuel_cleaned.csv"
DAILY_CSV = DATA_DIR / "germany_daily_by_fuel_MWh.csv"
MONTHLY_CSV = DATA_DIR / "germany_monthly_by_fuel_MWh.csv"
REN_HOURLY_CSV = DATA_DIR / "germany_hourly_renewable_nonrenewable.csv"
REN_DAILY_CSV  = DATA_DIR / "germany_daily_renewable_nonrenewable.csv"
REN_MONTHLY_CSV= DATA_DIR / "germany_monthly_renewable_nonrenewable.csv"
LONG_FORMAT_CSV= DATA_DIR / "germany_hourly_long.csv"

# Figures
FIG_STACKED_DAILY = DATA_DIR / "figure_stacked_area_daily_GWh.png"
FIG_REN_BAR = DATA_DIR / "figure_renewable_vs_nonrenewable_Oct_Dec_GWh.png"

In [ ]:
# --- 1. Acquire Data ---
# Strategy:
# 1) Try to load from local cache (./data/raw_generation.json).
# 2) If missing, try API (BASE_URL, API_KEY) – saves cache if successful.
# 3) If API unavailable, look for an already-downloaded file under ./data/:
#       - 'task_generation_h.csv'   (preferred if you exported from the provided link)
#       - 'task_generation_h.parquet'
#    If none exist, **download the dataset manually** from the shared link and place it in ./data/,
#    then re-run this cell.

def load_from_api():
    if not BASE_URL or not API_KEY:
        return None
    import requests
    headers = {"API-Key": API_KEY}
    params = {"dataset": DATASET, "from": DATE_FROM, "to": DATE_TO}
    r = requests.get(BASE_URL, headers=headers, params=params, timeout=60)
    r.raise_for_status()
    payload = r.json()
    # Expecting keys: "columns" and "data"
    df = pd.DataFrame(payload.get("data", []))
    if "columns" in payload and df.shape[1] == len(payload["columns"]):
        df.columns = payload["columns"]
    return df

def load_from_local_file():
    # Try Parquet or CSV that the candidate has placed under ./data
    pq = DATA_DIR / "task_generation_h.parquet"
    csv = DATA_DIR / "task_generation_h.csv"
    if pq.exists():
        return pd.read_parquet(pq)
    if csv.exists():
        return pd.read_csv(csv)
    return None

def load_data():
    # 1) local cache JSON
    if RAW_JSON_PATH.exists():
        try:
            obj = json.loads(RAW_JSON_PATH.read_text())
            df = pd.DataFrame(obj.get("data", []))
            if "columns" in obj and df.shape[1] == len(obj["columns"]):
                df.columns = obj["columns"]
            if not df.empty:
                print("Loaded cached API JSON:", RAW_JSON_PATH)
                return df
        except Exception as e:
            print("Cached JSON exists but could not be read:", e)

    # 2) API
    try:
        df_api = load_from_api()
        if df_api is not None and not df_api.empty:
            print("Loaded fresh data from API.")
            # Save cache
            cache_obj = {"columns": list(df_api.columns), "data": df_api.values.tolist()}
            RAW_JSON_PATH.write_text(json.dumps(cache_obj, indent=2))
            print("Saved API cache to", RAW_JSON_PATH)
            return df_api
    except Exception as e:
        print("API fetch failed:", repr(e))

    # 3) Local manual download
    df_local = load_from_local_file()
    if df_local is not None and not df_local.empty:
        print("Loaded local file from ./data")
        return df_local

    raise FileNotFoundError(
        "No data available. Please download the dataset from the provided link "
        "and place it as './data/task_generation_h.csv' (or .parquet), then re-run."
    )

df_raw = load_data()
df_raw.head()

In [ ]:
# --- 2. Filter for Germany & Pivot ---
# The raw schema is expected to contain: ['date_id', 'region', 'generation', 'value']
# We'll keep 'Germany', set 'date_id' as DatetimeIndex (hourly), and pivot to wide columns per fuel.

expected_cols = {"date_id","region","generation","value"}
missing = expected_cols - set(df_raw.columns)
if missing:
    raise ValueError(f"Input missing expected columns: {missing}")

df_raw["date_id"] = pd.to_datetime(df_raw["date_id"])
df_de = df_raw[df_raw["region"] == "Germany"].copy()
df_de = df_de.sort_values("date_id")

# Pivot: wide by generation (fuel type)
dfw = df_de.pivot_table(
    index="date_id",
    columns="generation",
    values="value",
    aggfunc="sum"  # in case there are duplicate rows, sum within the hour
).sort_index()

# (Optional) ensure hourly frequency if you want a full range with gaps visible
full_range = pd.date_range(dfw.index.min(), dfw.index.max(), freq="H")
dfw = dfw.reindex(full_range)  # keep NaN where data is missing; we will impute later
dfw.index.name = "date_id"

print("Wide hourly table (Germany) – shape:", dfw.shape)
dfw.head()

In [ ]:
# --- 3. Replace Deliberately Large Numbers (Outliers) ---
# We treat "implausibly large spikes" using a robust rule:
#   Flag values > Q3 + 1.5*IQR or < Q1 - 1.5*IQR (per-fuel).
# Replacement strategy:
#   Replace each outlier by the median for the *same month* and *same hour-of-day* for that fuel.
#   If that median is unavailable, fall back to the overall median for that fuel.

def month_hour_medians(s: pd.Series) -> pd.Series:
    # Returns a Series aligned to s.index with month-hour median values
    month = s.index.to_period("M")
    hour  = s.index.hour
    key = pd.MultiIndex.from_arrays([month, hour], names=["month","hour"])
    med = s.groupby(key).transform("median")
    # 'med' is aligned to s by construction
    return med

def replace_outliers_iqr(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    for col in df.columns:
        s = df[col]
        q1, q3 = s.quantile(0.25), s.quantile(0.75)
        iqr = q3 - q1
        if pd.isna(iqr) or iqr == 0:
            # Not enough variation; skip
            continue
        fence_low  = q1 - 1.5 * iqr
        fence_high = q3 + 1.5 * iqr
        mask = (s < fence_low) | (s > fence_high)
        if mask.any():
            mh_med = month_hour_medians(s)
            fallback = s.median()
            repl = mh_med.where(~mh_med.isna(), other=fallback)
            df.loc[mask, col] = repl[mask]
    return df

dfw_no_outliers = replace_outliers_iqr(dfw)
dfw_no_outliers.head()

In [ ]:
# --- 4. Impute Missing Data (Intentional Gaps) ---
# Two-step imputation per fuel:
#   1) Time-based linear interpolation (works well for hourly series).
#   2) Fill any remaining gaps with month-hour median; fallback to overall median.

def impute_missing(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # Step 1: linear interpolation over time
    df = df.interpolate(method="time", limit_direction="both")

    # Step 2: month-hour median per column
    for col in df.columns:
        s = df[col]
        still_na = s.isna()
        if still_na.any():
            mh_med = month_hour_medians(s)
            s.loc[still_na] = mh_med.loc[still_na]
            # final fallback
            still_na = s.isna()
            if still_na.any():
                s.loc[still_na] = s.median()
        df[col] = s
    return df

dfw_clean = impute_missing(dfw_no_outliers)
print("Any remaining NaNs?", dfw_clean.isna().sum().sum())
dfw_clean.head()

In [ ]:
# --- 5. Aggregate Daily & Monthly by Fuel ---
# IMPORTANT: The raw is hourly MWh. Aggregation should be **sum**, not mean.
# We compute daily and monthly totals per fuel (MWh).

df_daily = dfw_clean.resample("D").sum()
df_monthly = dfw_clean.resample("M").sum()

print("Daily shape:", df_daily.shape, "Monthly shape:", df_monthly.shape)
df_daily.head()

In [ ]:
# --- 6. BONUS: Renewable vs Non-Renewable ---
# Map each fuel type to one of two categories, then compute hourly, daily, monthly
# totals and percentages.

renewable_sources = [
    "Biomass", "Dam Hydro", "Geothermal", "Other renewables",
    "Pumped storage generation", "Run-of-River Hydro",
    "Solar", "Wind offshore", "Wind onshore"
]

non_renewable_sources = [
    "Hard Coal", "Lignite", "Natural Gas", "Non-renewable waste",
    "Nuclear", "Oil", "Other fossil fuel"
]

# Keep only columns present in the dataset
ren_cols = [c for c in renewable_sources if c in dfw_clean.columns]
nren_cols = [c for c in non_renewable_sources if c in dfw_clean.columns]

hourly_ren = pd.DataFrame({
    "renewable_MWh": dfw_clean[ren_cols].sum(axis=1) if ren_cols else 0.0,
    "non_renewable_MWh": dfw_clean[nren_cols].sum(axis=1) if nren_cols else 0.0,
})
hourly_ren["total_MWh"] = hourly_ren.sum(axis=1)
hourly_ren["renewable_%"] = np.where(hourly_ren["total_MWh"]>0, 100*hourly_ren["renewable_MWh"]/hourly_ren["total_MWh"], np.nan)
hourly_ren["non_renewable_%"] = np.where(hourly_ren["total_MWh"]>0, 100*hourly_ren["non_renewable_MWh"]/hourly_ren["total_MWh"], np.nan)

daily_ren = hourly_ren.resample("D").sum()
daily_ren["renewable_%"] = np.where(daily_ren["total_MWh"]>0, 100*daily_ren["renewable_MWh"]/daily_ren["total_MWh"], np.nan)
daily_ren["non_renewable_%"] = np.where(daily_ren["total_MWh"]>0, 100*daily_ren["non_renewable_MWh"]/daily_ren["total_MWh"], np.nan)

monthly_ren = hourly_ren.resample("M").sum()
monthly_ren["renewable_%"] = np.where(monthly_ren["total_MWh"]>0, 100*monthly_ren["renewable_MWh"]/monthly_ren["total_MWh"], np.nan)
monthly_ren["non_renewable_%"] = np.where(monthly_ren["total_MWh"]>0, 100*monthly_ren["non_renewable_MWh"]/monthly_ren["total_MWh"], np.nan)

hourly_ren.head()

In [ ]:
# --- 7. BONUS: Back to Long Format ---
# Convert the **cleaned hourly wide** table back to the original long schema:
# ['date_id','region','generation','value'] for Germany only.

df_long = (
    dfw_clean
      .reset_index()
      .melt(id_vars="date_id", var_name="generation", value_name="value")
      .assign(region="Germany")
      .loc[:, ["date_id","region","generation","value"]]
      .sort_values(["date_id","generation"])
      .reset_index(drop=True)
)
df_long.head()

In [ ]:
# --- 8. Plots (Matplotlib) ---
# 1) Stacked area of **daily** totals by fuel (GWh).
# 2) (Bonus) Bar chart comparing Renewable vs Non-Renewable in Oct and Dec 2022 (GWh).

# 1) Stacked area (daily by fuel)
daily_gwh = df_daily / 1000.0  # MWh -> GWh
ax = daily_gwh.plot.area()
ax.set_title("Germany – Daily Power Generation by Fuel (GWh)")
ax.set_xlabel("Date")
ax.set_ylabel("GWh")
plt.tight_layout()
plt.savefig(FIG_STACKED_DAILY, dpi=150)
plt.show()

# 2) Renewable vs Non-Renewable for Oct & Dec 2022
oct_mask = (daily_ren.index >= "2022-10-01") & (daily_ren.index <= "2022-10-31")
dec_mask = (daily_ren.index >= "2022-12-01") & (daily_ren.index <= "2022-12-31")

oct_totals = daily_ren.loc[oct_mask, ["renewable_MWh","non_renewable_MWh"]].sum()/1000.0
dec_totals = daily_ren.loc[dec_mask, ["renewable_MWh","non_renewable_MWh"]].sum()/1000.0

fig = plt.figure()
x = np.arange(2)
width = 0.35
plt.bar(x - width/2, [oct_totals["renewable_MWh"], dec_totals["renewable_MWh"]], width, label="Renewable")
plt.bar(x + width/2, [oct_totals["non_renewable_MWh"], dec_totals["non_renewable_MWh"]], width, label="Non-Renewable")
plt.xticks(x, ["Oct 2022", "Dec 2022"])
plt.ylabel("GWh")
plt.title("Germany – Renewable vs Non-Renewable (Oct & Dec 2022, GWh)")
plt.legend()
plt.tight_layout()
plt.savefig(FIG_REN_BAR, dpi=150)
plt.show()

In [ ]:
# --- 9. Save Outputs ---
dfw_clean.to_csv(CLEAN_HOURLY_WIDE_CSV, index=True)
df_daily.to_csv(DAILY_CSV, index=True)
df_monthly.to_csv(MONTHLY_CSV, index=True)

hourly_ren.to_csv(REN_HOURLY_CSV, index=True)
daily_ren.to_csv(REN_DAILY_CSV, index=True)
monthly_ren.to_csv(REN_MONTHLY_CSV, index=True)

df_long.to_csv(LONG_FORMAT_CSV, index=False)

print("Saved:")
print(" -", CLEAN_HOURLY_WIDE_CSV)
print(" -", DAILY_CSV)
print(" -", MONTHLY_CSV)
print(" -", REN_HOURLY_CSV)
print(" -", REN_DAILY_CSV)
print(" -", REN_MONTHLY_CSV)
print(" -", LONG_FORMAT_CSV)
print(" -", FIG_STACKED_DAILY)
print(" -", FIG_REN_BAR)

# --- 10. Notes & Assumptions (for reviewers) ---
# • Aggregation uses **sum** (MWh → daily/monthly totals). The original draft used mean;
#   totals are the typical requirement for energy generation.
# • Outliers: substituted with the month × hour-of-day median per fuel – a "reasonable" value
#   that respects diurnal and seasonal patterns without using future information.
# • Missing values: linear time interpolation first; remaining gaps filled with month × hour-of-day
#   medians, then overall median fallback.
# • Renewable mapping follows the provided grouping. Only fuels present in the dataset are used.
# • All outputs are saved under ./data for easy inspection and re-use by a frontend later.
# • This notebook is self-contained: can run with API credentials or an offline CSV/Parquet in ./data.